In [25]:
!pip install pyTelegramBotAPI

In [26]:
import telebot
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img

In [27]:
bot = telebot.TeleBot("7739012435:AAFm0454ePnSA1Cja1v28HCO5AWYST4yF_k", parse_mode="HTML")
model = load_model('/content/my_model.h5')

In [29]:

class_dict={'Akhund': 0, 'Human': 1}

datagen = ImageDataGenerator(rescale=1./255)

@bot.message_handler(commands=['start'])
def send_welcome(message):
    bot.reply_to(message, f"Hi {message.from_user.first_name}!  Send me a picture.")

@bot.message_handler(content_types=['photo'])
def handle_photo(message):
    file_id = message.photo[-1].file_id
    file_info = bot.get_file(file_id)
    downloaded_file = bot.download_file(file_info.file_path)

    with open("image.jpg", "wb") as file:
        file.write(downloaded_file)

    bot.reply_to(message, "🔄 Processing the image...")

    image = load_img("image.jpg", target_size=(224, 224))
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)

    image_gen = datagen.flow(image, batch_size=1)
    image_processed = next(image_gen)

    output = model.predict(image_processed)
    predict = np.argmax(output)
    print(predict)
    class_name = class_dict.get(predict, "Unknown about this")

    if predict == 0:
      text = "از ما نیست ! 😑"
    elif predict == 1:
      text = "از ماست 😍"
    else:
      text = "شناخته نشد 😐"

    bot.reply_to(message,f"{text}")

bot.polling()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1
